In [1]:
# coding:utf-8

import pandas as pd

xgb100 = pd.read_csv('../datas/xgb_40_filter_yval_pred')
adboost80 = pd.read_csv('../datas/adboost_80_filter_yval_pred')
lgb80 = pd.read_csv('../datas/lgb80_filter_yval_pred')

In [2]:
xgb100 = xgb100.sort_values('y',ascending=False).reset_index(drop=True)
adboost80 = adboost80.sort_values('y',ascending=False).reset_index(drop=True)
lgb80 = lgb80.sort_values('y',ascending=False).reset_index(drop=True)

In [3]:
# 融合方法一
bs = set(xgb100.loc[:20]['rowkey'].values.tolist() + \
    adboost80.loc[:25]['rowkey'].values.tolist() + \
    lgb80.loc[:12]['rowkey'].values.tolist())

In [6]:
#xgb100['rowkey'].map(lambda x:x in bs)

In [26]:
d1 = xgb100[xgb100['rowkey'].map(lambda x:x in bs)]
d2 = adboost80[adboost80['rowkey'].map(lambda x:x in bs)]
d2.columns = ['is_risk','rowkey','y2']
d3 = lgb80[lgb80['rowkey'].map(lambda x:x in bs)]
d3.columns = ['is_risk','rowkey','y3']

d1.head()

,is_risk,rowkey,y
0,1,755883,0.979718
1,1,759124,0.932931
2,1,758150,0.918475
3,1,757002,0.911895
4,1,759540,0.905213


In [52]:
d['a'] = -1
d['a'] += d['rowkey'].map(lambda x:1 if x==759124 else 0)
d['a'] += d['yy'].map(lambda x:1 if x>=0.924840 else 0)

d.head()

,is_risk,rowkey,y,y2,y3,yy,yymin,yymax,a
0,1,759124,0.932931,0.509323,0.984288,1.442254,0.509323,0.984288,1
1,1,757002,0.911895,0.504133,0.924840,1.416028,0.504133,0.924840,0
2,1,756804,0.902701,0.502972,0.896534,1.399506,0.502972,0.902701,0
3,1,758150,0.918475,0.504544,0.864071,1.368615,0.504544,0.918475,0
4,1,759540,0.905213,0.512862,0.851826,1.364688,0.512862,0.905213,0


In [27]:
d2.head()

,is_risk,rowkey,y2
0,0,884465,0.526404
1,1,929869,0.516687
2,1,757171,0.515470
3,1,785025,0.515455
4,1,737666,0.513542


In [28]:
d = d1.merge(d2[['rowkey','y2']],on='rowkey')
d = d.merge(d3[['rowkey','y3']],on='rowkey')

d.head()

,is_risk,rowkey,y,y2,y3
0,1,755883,0.979718,0.501715,0.835978
1,1,759124,0.932931,0.509323,0.984288
2,1,758150,0.918475,0.504544,0.864071
3,1,757002,0.911895,0.504133,0.924840
4,1,759540,0.905213,0.512862,0.851826


In [44]:
d['yymin'] = d[['y','y2','y3']].min(axis=1)
d['yymax'] = d[['y','y2','y3']].max(axis=1)
d['yy'] = d[['y','y2','y3']].sum(axis=1) - d['yymax']

In [45]:
d = d.sort_values('yy',ascending=False).reset_index(drop=True)
d

,is_risk,rowkey,y,y2,y3,yy,yymin,yymax
0,1,759124,0.932931,0.509323,0.984288,1.442254,0.509323,0.984288
1,1,757002,0.911895,0.504133,0.924840,1.416028,0.504133,0.924840
2,1,756804,0.902701,0.502972,0.896534,1.399506,0.502972,0.902701
3,1,758150,0.918475,0.504544,0.864071,1.368615,0.504544,0.918475
4,1,759540,0.905213,0.512862,0.851826,1.364688,0.512862,0.905213
5,1,737666,0.882007,0.513542,0.829649,1.343192,0.513542,0.882007
6,1,758749,0.841781,0.496850,0.908591,1.338631,0.496850,0.908591
7,1,755883,0.979718,0.501715,0.835978,1.337693,0.501715,0.979718
8,1,758960,0.883762,0.497488,0.827948,1.325436,0.497488,0.883762
9,1,785025,0.763625,0.515455,0.818648,1.279080,0.515455,0.818648
